# Project Milestone 2

In [4]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install pandas==1.0.5

     |████████████████████████████████| 10.1 MB 5.0 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.0.5 which is incompatible.


In [1]:
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import bz2
import json
from urllib.parse import urlparse
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import os
import glob
import pickle

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Part 1: Filtering

## Create Dataframe for each journal 


In [2]:
def process_chunk(chunk, year, nb, df):
  
    df = chunk.drop(['quoteID', 'qids', 'numOccurrences', 'probas', 'phase'], axis=1) #axis=1 for columns
    
    df['urls'] = df['urls'].astype('str') #Converting to string to be able to use str.contains 
    df_1 = df[df['urls'].str.contains('foxnews')] #Creating a DataFrame containing only foxnews
    df_2 = df[df['urls'].str.contains('nytimes')]
    df = pd.concat([df_1, df_2])

    df.to_pickle('/content/drive/MyDrive/ADA_2021/' + str(nb)+ '_'+  str(year) + '_' + 'FoxNYtimes.pkl')
    #df.to_pickle("/content/drive/MyDrive/ADA_2021/Fox_NY_"+ str(year) + '/' + str(nb)+ '_'+  str(year) + '_' + "FoxNYtimes.pkl")

In [ ]:
years = [2015, 2016, 2017, 2018, 2019, 2020]
nb = 1
df = pd.DataFrame()
for y in years:
    for chunk in pd.read_json('/content/drive/MyDrive/Quotebank/quotes-' + str(year) + '.json.bz2', 
                              lines=True, compression='bz2', chunksize=500000):
      
      process_chunk(chunk, y, nb, df)
      nb += 1

In [3]:
def read_yearly_chunk(year):
    PATH = '/content/drive/MyDrive/ADA_2021/Fox_NY_' + str(year) + '/'
    nb = 1
    df2 = []
    #dirs = os.listdir(PATH)
    dirs = glob.glob(os.path.join(PATH, "*.pkl"))
    
    #for file in dirs:
    for files in dirs:
        df1 = pd.read_pickle(PATH + str(nb)+ '_' + str(year) + '_' + 'FoxNYtimes.pkl')
        df2.append(df1)
        nb += 1
    return df2

In [5]:
years = [2015, 2016, 2017, 2018, 2019, 2020]
df_FoxNY = pd.DataFrame()

for y in years:
  df_FoxNY = df_FoxNY.append(pd.concat(read_yearly_chunk(y)))

In [6]:
df_FoxNY

,quotation,speaker,date,urls
314,Coach is an awesome dude to play for. He is 10...,Eric Fisher,2020-01-28 12:30:44,['https://www.seattletimes.com/sports/nba/chie...
409,"Finally, we reached the point a few weeks ago ...",Elizabeth Warren,2020-02-20 00:00:00,['http://feeds.foxnews.com/~r/foxnews/politics...
421,"For example, multiple employees recalled a per...",None,2020-02-07 00:00:00,['http://www.foxnews.com/media/new-york-times-...
458,Given the partisan nature of this impeachment ...,Lisa Murkowski,2020-01-31 00:00:00,['http://feeds.foxnews.com/~r/foxnews/politics...
516,He was someone you could lean on.,Mark Dantonio,2020-01-08 00:00:00,['https://www.foxnews.com/sports/george-perles...
...,...,...,...,...
5244002,trying to understand my own culture.,None,2020-02-07 15:00:10,['http://mobile.nytimes.com/2020/02/07/arts/mu...
5244135,"We Hanks' will be tested, observed, and isolat...",Tom Hanks,2020-03-11 00:00:00,['http://www.dailyherald.com/news/20200311/tom...
5244172,we immediately initiated an investigation and ...,None,2020-01-19 18:08:35,['http://mobile.nytimes.com/2020/01/19/world/a...
5244252,"Well, Joe, you're wrong.",Bernie Sanders,2020-01-26 17:01:39,['https://www.nytimes.com/live/2020/iowa-democ...


### Sentimental Analysis